In [ ]:
import sys
from PIL import Image
import sklearn
import matplotlib
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.utils import load_img, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import cv2
import pandas as pd

In [ ]:
# !pip install imbalanced-learn

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu huấn luyện và kiểm tra
train_dir = '/content/drive/MyDrive/Các kì học tại HUS/B. Học kì 6 tại HUS/Seminar /Nghiên cứu/archive/train/'
test_dir = '/content/drive/MyDrive/Các kì học tại HUS/B. Học kì 6 tại HUS/Seminar /Nghiên cứu/archive/test/'

CATEGORIES = ['angry','disgust','fear','happy','neutral', 'sad', 'surprise']

In [ ]:
training_data=[]
def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(train_dir, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                img_gray = cv2.cvtColor(img_array,cv2.COLOR_BGR2GRAY)
                new_array=cv2.resize(img_gray,(48,48))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Các kì học tại HUS/B. Học kì 6 tại HUS/Seminar /Nghiên cứu/archive/train/angry'

In [ ]:
test_data=[]
def create_test_data():
    for category in CATEGORIES:
        path=os.path.join(test_dir, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                img_gray = cv2.cvtColor(img_array,cv2.COLOR_BGR2GRAY)
                new_array=cv2.resize(img_gray,(48,48))
                test_data.append([new_array,class_num])
            except Exception as e:
                pass
create_test_data()

In [ ]:
x_train=[]
y_train=[]

for categories, label in training_data:
    x_train.append(categories)
    y_train.append(label)

In [ ]:
x_test=[]
y_test=[]

for categories, label in test_data:
    x_test.append(categories)
    y_test.append(label)

In [ ]:
# preprocessing the images
# Chuyển đổi số chiều của X_train và X_test
X_train = np.array(x_train).reshape(len(x_train),-1)
X_test = np.array(x_test).reshape(len(x_test),-1)
# Chuyển đổi giá trị pixel thành đoạn [0, 1]
X_train =  X_train.astype(float) / 255.0
X_test =  X_test.astype(float) / 255.0


print(X_train.shape)
print(X_train[0].shape)
X_train

(28694, 2304)
(2304,)


array([[0.4627451 , 0.4627451 , 0.55686275, ..., 0.37647059, 0.38039216,
        0.38823529],
       [0.51372549, 0.52156863, 0.51372549, ..., 0.67843137, 0.70196078,
        0.69411765],
       [0.3254902 , 0.34117647, 0.34509804, ..., 0.27843137, 0.26666667,
        0.31764706],
       ...,
       [0.36862745, 0.39215686, 0.44705882, ..., 0.59215686, 0.62745098,
        0.67058824],
       [0.17254902, 0.19215686, 0.1372549 , ..., 0.83529412, 0.65882353,
        0.50196078],
       [0.98431373, 1.        , 0.82745098, ..., 0.98039216, 1.        ,
        1.        ]])

In [ ]:
# Chuyển đổi y_train và y_test thành mảng 1D và kiểu dữ liệu int64
y_train = np.array(y_train)
print(y_train.shape)
y_test = np.array(y_test)
print(y_test.shape)

(28694,)
(7178,)


In [ ]:
y_test

array([0, 0, 0, ..., 6, 6, 6])

In [ ]:
from imblearn.over_sampling import SMOTE

# Tạo đối tượng SMOTE
smote = SMOTE()

# Áp dụng Oversampling bằng SMOTE cho dữ liệu huấn luyện
X_train, y_train = smote.fit_resample(X_train, y_train)

# Kiểm tra kích thước của dữ liệu sau khi Oversampling
print("Kích thước dữ liệu sau khi Oversampling:", X_train.shape, y_train.shape)

Kích thước dữ liệu sau khi Oversampling: (50498, 2304) (50498,)


In [ ]:
# Đếm số lượng mẫu của từng nhãn
unique_labels, counts = np.unique(y_train, return_counts=True)

for label, count in zip(unique_labels, counts):
    print("Nhãn", label, ": Số lượng mẫu =", count)

Nhãn 0 : Số lượng mẫu = 7214
Nhãn 1 : Số lượng mẫu = 7214
Nhãn 2 : Số lượng mẫu = 7214
Nhãn 3 : Số lượng mẫu = 7214
Nhãn 4 : Số lượng mẫu = 7214
Nhãn 5 : Số lượng mẫu = 7214
Nhãn 6 : Số lượng mẫu = 7214


# **RANDOM FOREST**

In [ ]:
# #Đánh giá và cải thiện mô hình bằng cách sử dụng các kỹ thuật như cross-validation,
# #tối ưu hóa siêu tham số và đánh giá mô hình trên tập kiểm tra.
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
# from sklearn.metrics import classification_report

# # Define the model
# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=100)

# # Số lượng fold và khởi tạo KFold
# k = 10
# kf = KFold(n_splits=k, shuffle=True, random_state=42)

# # Grid search for hyperparameter tuning
# params = {'n_estimators': [100, 200, 300, 400, 500], 'max_depth': [2,3,5,10,20], 'min_samples_leaf': [2,5,10,20,50,100,200]}
# grid_search = GridSearchCV(model, params, cv=kf, scoring='accuracy')
# grid_search.fit(X_train, y_train)
# print('Best hyperparameters:', grid_search.best_params_)


In [ ]:
# # Xây dựng mô hình SVM với giá trị tốt nhất của các hyperparameter
# best_model = grid_search.best_estimator_
# best_model.fit(X_train, y_train)

# # Dự đoán nhãn cho tập kiểm tra
# y_pred = best_model.predict(X_test)
# print(classification_report(y_test, y_pred))
# # Đánh giá mô hình bằng độ chính xác
# accuracy = accuracy_score(y_test, y_pred)
# print("Độ chính xác của mô hình SVM:", accuracy)

In [ ]:
# # Chạy cross-validation và tính độ chính xác và độ lệch chuẩn
# scores = cross_val_score(best_model, X, y, cv=10, scoring='accuracy')

# print("Mean Accuracy (after hyperparameter tuning):", scores.mean())
# print("Standard Deviation (after hyperparameter tuning):", scores.std())

Best model

In [ ]:
#Xây dựng một mô hình Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import time

model = RandomForestClassifier(n_estimators=500, max_depth=10, min_samples_leaf = 2, random_state=100)
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
y_pred = model.predict(X_test)

In [ ]:
y_pred.shape

(7178,)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

# Tính thời gian huấn luyện mô hình
training_time = end_time - start_time
print("Thời gian huấn luyện mô hình: {:.2f} giây".format(training_time))

Accuracy: 0.39816104764558374
              precision    recall  f1-score   support

           0       0.32      0.24      0.28       958
           1       0.20      0.50      0.29       111
           2       0.32      0.22      0.26      1024
           3       0.56      0.50      0.53      1774
           4       0.39      0.34      0.36      1233
           5       0.34      0.38      0.36      1247
           6       0.39      0.69      0.50       831

    accuracy                           0.40      7178
   macro avg       0.36      0.41      0.37      7178
weighted avg       0.40      0.40      0.39      7178

Thời gian huấn luyện mô hình: 789.56 giây


In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[230,  24,  92, 168, 122, 173, 149],
       [ 10,  55,   4,  18,   5,   8,  11],
       [ 85,  24, 224, 141, 109, 191, 250],
       [102,  51, 100, 885, 199, 261, 176],
       [ 96,  39, 108, 166, 416, 235, 173],
       [147,  59, 133, 145, 161, 474, 128],
       [ 40,  18,  44,  52,  50,  53, 574]])